In [ ]:
using Pkg
Pkg.add(url="https://github.com/toubinaattori/thesis")
using DecisionProgramming
using JuMP, Distributions

N = 6
h = rand(20)
plibt = rand(20)
for xibiz in 1:20
    diagram = InfluenceDiagram()
    probs_false_positive = rand(N)
    probs_false_negative = rand(N)
    for pob in 1:N
        mean = sort(rand(2))
        sigma = rand(2)
        d1 = Normal(mean[1], sigma[1])
        td1 = truncated(d1, 0.0, Inf)
        negative = rand(td1, 100)

        d2 = Normal(mean[2], sigma[2])
        td2 = truncated(d2, 0.0, Inf)
        positive = rand(td2, 100)

        i = 1000
        cutoff = 0
        for x in sort(vcat(positive, negative))
            falses = sum(map(y -> y < x, positive)) + sum(map(y -> y >= x, negative))
            if falses < i
                i = falses
                cutoff = x
            end
        end
        probs_false_positive[pob] = sum(map(y -> y < cutoff, positive))/size(positive)[1]
        probs_false_negative[pob] = sum(map(y -> y >= cutoff, negative))/size(negative)[1]
    end
    
    
    
    costs_initial = sort(rand(0:50, N),rev=true)
    costs = rand(N)
    false_p = sort(probs_false_positive)
    for i in 1:N
        s = findall(x-> x == false_p[i] , probs_false_positive)
        costs[s[1]] = costs_initial[i]
    end

    diagram.Cost = []

    add_node!(diagram, ChanceNode("O", [], [ "yes", "no"]))

    for i in 1:N
        add_node!(diagram, ChanceNode("R$i", ["O"], ["yes", "no"]))
    end

    add_node!(diagram, DecisionNode("D", map(x -> "R$x", 1:N), ["yes", "no"],map(x -> "R$x", 1:N)))
    add_node!(diagram, ValueNode("V", ["O","D"]))

    for i in 1:N
        add_costs!(diagram,Costs(("R$i","D"), costs[i]))
    end


    generate_arcs!(diagram)

    add_probabilities!(diagram, "O", [0.15,0.85])
    for i in 1:N
        X_R = ProbabilityMatrix(diagram, "R$i")
        X_R["yes","yes"] = 1- probs_false_negative[i]
        X_R["no","yes"] =  probs_false_positive[i]
        X_R["no","no"] = 1- probs_false_positive[i]
        X_R["yes","no"] = probs_false_negative[i]
        add_probabilities!(diagram, "R$i", X_R)
    end

    Y_V = UtilityMatrix(diagram, "V")
    Y_V["yes", "yes"] = 6700
    Y_V["yes","no"] = 0
    Y_V["no","yes"] = -300
    Y_V["no","no"] = 0
    add_utilities!(diagram, "V", Y_V)

    generate_diagram!(diagram)

    model = Model()
    z = DecisionVariables(model, diagram)
    x_s = PathCompatibilityVariables(model, diagram, z, names=true, name = "s")
    x_x = ActiveDecisionPathVariables(model,diagram,z,x_s,names=true,name="x")
    EV = expected_value(model, diagram, x_s,x_x)

    @objective(model, Max, EV)

    using GLPK
    set_optimizer(model, GLPK.Optimizer)
    optimize!(model)
    h[xibiz] = solve_time(model)
    pablo = 0
    for x_x_x in x_x
        pablo = pablo + value.(x_x_x[2])
    end
    plibt[xibiz] = pablo
end
println(sum(h)/20)
println(sum(plibt)/20)